In [ ]:
import numpy as np
%load_ext autoreload
%autoreload 2

In [ ]:
import mesa
from ice_fishing_m1.model import IceFishingModel

In [ ]:
m1 = IceFishingModel()

In [ ]:
m1.run_model(step_count=1000)

### Check the data

In [ ]:
results = m1.datacollector.get_model_vars_dataframe()

In [ ]:
results

## Batch run three environments

In [ ]:
%%time
from ice_fishing_m1.model import IceFishingModel
params = {"width": 30, "height": 30, "fish_patch_n_samples": (500, 2000, 10_000), "agent_model": ("random", "imitator", "greedy_bayesian")}

results = mesa.batch_run(
    IceFishingModel,
    parameters=params,
    iterations=100,
    max_steps=1000,
    number_processes=8,
    data_collection_period=1,
    display_progress=True,
)



In [ ]:
import pandas as pd
import matplotlib.pylab as plt

results_df = pd.DataFrame(results)
print(results_df.keys())

In [ ]:
results_df.boxplot(column='Mean catch ratio', by=['fish_patch_n_samples', 'agent_model'], grid=False, rot=90)
plt.xlabel('Source smoothness & agent model')
plt.ylabel('Mean catch ration')
plt.suptitle(None)
plt.title('Catch in 1000 steps')
plt.show();

## Run different resource cluster std and smoothness

In [ ]:
%%time
import numpy as np
from ice_fishing_m1.model import IceFishingModel

n_steps = 1000

params = {
    "width": 30,
    "height": 30,
    "fish_patch_n_samples": 1000,  # (500, 2000, 10_000),
    "fish_patch_std": np.linspace(0.2, 1, 3),
    "agent_model": ("random", "imitator", "greedy_bayesian")
}

results = mesa.batch_run(
    IceFishingModel,
    parameters=params,
    iterations=100,
    max_steps=n_steps,
    number_processes=8,
    data_collection_period=1,
    display_progress=True,
)



In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pylab as plt

results_df = pd.DataFrame(results)
print(results_df.keys())

In [ ]:
# set beautiful sns style
sns.set_theme(style="white")

sns.lineplot(
    data=results_df[results_df['Step'] == n_steps],
    x='fish_patch_std', y='Mean catch ratio', hue='agent_model')

sns.despine()

# add labels
plt.xlabel('Resource cluster smoothness')
plt.ylabel('Mean catch ration')

# legend title
plt.legend(title='Agent model')

plt.show();

In [ ]:
# select the last step
results_last_step_df = results_df[results_df['Step'] == 1000]

# round patch std to 2 decimals
results_last_step_df.loc[:, 'fish_patch_std'] = results_last_step_df['fish_patch_std'].round(2)

# average over the iterations
results_last_step_df = results_last_step_df.groupby(['fish_patch_std', 'fish_patch_n_samples', 'agent_model']).mean().reset_index()

results_last_step_df

In [ ]:
# df_random = results_last_step_df[results_last_step_df['agent_model'] == 'random']
# pivot results for two models independently

heatmap_df = results_last_step_df.pivot(
    index='agent_model', columns='fish_patch_std', values='Mean catch ratio')

sns.heatmap(heatmap_df, cmap="viridis", vmin=0,#  vmax=500,
            cbar_kws={'label': 'Mean catch ratio'})

# add title
# plt.title('Random agent')

plt.show();